In [1]:
import keras
import jieba
import logging
import re
import gensim
import pandas as pd

Using TensorFlow backend.


In [2]:
jieba.setLogLevel(logging.INFO)#显示结巴分词日志

regex = re.compile(r'[^\u4e00-\u9fa5aA-Za-z0-9]')#正则表达式，返回的是一个匹配对象，https://www.runoob.com/regexp/regexp-syntax.html

# def word_cut(text):#切的结果为字符串
#     text = regex.sub('', text)
#     cut_str = ' '.join(jieba.cut(text, cut_all=False))#切为字符串并在词间穿插空格
#     return cut_str

def word_cut(text):#结果为list
    text = regex.sub(' ', text)
    return [word for word in jieba.cut(text) if word.strip()]



In [3]:
train = pd.read_csv('data/test.tsv', sep='\t',)
dev = pd.read_csv('data/dev.tsv', sep='\t',)
x_all = list(pd.read_csv("data/ch_auto.csv")["text"])
#train.head(5)#查看数据样式

train_x = train['text']#按行抽取
train_y = train['label']
dev_x = dev['text']
dev_y = dev['label']

train_dataset = []#将组合好的(x，y)放入此列表中
dev_dataset = []

for i in range(len(train_x)):#整合为训练数据集（初步）
    train_dataset.append((train_x[i],train_y[i]))
print("训练集：",train_dataset[:5])
print("-"*50)
for i in range(len(dev_x)):#整合为验证数据集（初步）
    dev_dataset.append((dev_x[i],dev_y[i]))
print("验证集：",dev_dataset[:5])

训练集： [('操控性舒服、油耗低，性价比高', 1), ('动力的确有点点让我相信了up的确是个代步车而已!', 0), ('1。车的外观很喜欢。2。省油，现在磨合期7.3，相信以后还会下降。', 1), ('内饰的做工和用料同级别同价位最厚道的', 1), ('减震系统太硬！', 0)]
--------------------------------------------------
验证集： [('外观确实非常霸气，钻石切工有棱有角，3.0的动力在城市里绰绰有余，内饰考究，空间比较大，bose的音响非常给力，小众品牌不像德系三架马车那样成为街车，为个性代言。', 1), ('外观漂亮，安全性佳，动力够强，油耗够低', 1), ('后备箱大！！！', 1), ('空间大。外观大气，中控台用料讲究简洁', 1), ('外观漂亮，空间够大，动力家用也ok', 1)]


In [4]:
#将训练集及验证集的x整合到一起制作词向量训练原始数据

#x_all = list(train_x)+list(dev_x)
#print("长度检查：",len(train_x)+len(dev_x)==len(x_all))#检查长度

In [5]:
#分词
x_all_cut=[]#分词后的语料库
for i in range(len(x_all)):
    x_all_cut.append(word_cut(x_all[i]))
print(len(x_all_cut))

70000


In [6]:
#使用gensim训练词向量并保存至本地
word_model = gensim.models.Word2Vec(x_all_cut,min_count=1)
word_model.save("word_model/word2vec.model")

In [7]:
#加载并测试词向量
#word_model["人性化"]
word_model.similarity("亮点","特性")#计算词汇相似度测试
#word_model.n_similarity()

/Users/fgd/.conda/envs/keras-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


0.5148908

In [8]:
word_model["儿子"].shape

/Users/fgd/.conda/envs/keras-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(100,)

In [9]:
import numpy as np
def build_sentence_vector(sentence,size,w2v_model):
    sen_vec=np.zeros(size).reshape((1,size))
    count=0
    for word in sentence:
        try:
            sen_vec+=w2v_model[word].reshape((1,size))
            count+=1
        except KeyError:
            continue
    if count!=0:
        sen_vec/=count
    return sen_vec


In [10]:
# result = build_sentence_vector(train_x[0],100,word_model)
# print(result.shape)

(1, 100)


/Users/fgd/.conda/envs/keras-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [11]:
def generate_id2wec(model):
    gensim_dict = gensim.corpora.dictionary.Dictionary()
    gensim_dict.doc2bow(model.wv.vocab.keys(), allow_update=True)
    w2id = {v: k + 1 for k, v in gensim_dict.items()}  # 词语的索引，从1开始编号
    w2vec = {word: model[word] for word in w2id.keys()}  # 词语的词向量
    n_vocabs = len(w2id) + 1
    embedding_weights = np.zeros((n_vocabs, 100))
    for w, index in w2id.items():  # 从索引为1的词语开始，用词向量填充矩阵
        embedding_weights[index, :] = w2vec[w]
    return w2id,embedding_weights


def text_to_array(w2index, senlist):  # 文本转为索引数字模式
    sentences_array = []
    for sen in senlist:
        new_sen = [ w2index.get(word,0) for word in sen]   # 单词转索引数字
        sentences_array.append(new_sen)
    return np.array(sentences_array)


def prepare_data(w2id,sentences,labels,max_len=100):
    X_train, X_val, y_train, y_val = train_test_split(sentences,labels, test_size=0.2)
    X_train = text_to_array(w2id, X_train)
    X_val = text_to_array(w2id, X_val)
    X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
    X_val = keras.preprocessing.sequence.pad_sequences(X_val, maxlen=max_len)
    return np.array(X_train), keras.utils.np_utils.to_categorical(y_train) ,np.array(X_val), keras.utils.np_utils.to_categorical(y_val)

In [12]:
w2id,embedding_weights = generate_id2wec(word_model)

/Users/fgd/.conda/envs/keras-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [13]:
from sklearn.model_selection import train_test_split
sentences = list(train_x)+list(dev_x)
labels = list(train_y)+list(dev_y)
for i in range(len(sentences)):
    sentences[i] = word_cut(sentences[i])
sentences[:5]
x_train,y_train, x_val , y_val = prepare_data(w2id,sentences,labels,200)


In [20]:
from keras.layers import Bidirectional,LSTM,Dense,Embedding,Dropout,Activation,Softmax

class Sentiment:
    def __init__(self,w2id,embedding_weights,Embedding_dim,maxlen,labels_category):
        self.Embedding_dim = Embedding_dim
        self.embedding_weights = embedding_weights
        self.vocab = w2id
        self.labels_category = labels_category#类别数量
        self.maxlen = maxlen
        self.model = self.build_model()#模型通过build_model方法来构建
      
        
    def build_model(self):
        model = keras.Sequential()
        #input dim(140,100)
        model.add(Embedding(output_dim = self.Embedding_dim,
                           input_dim=len(self.vocab)+1,
                           weights=[self.embedding_weights],
                           input_length=self.maxlen))
        model.add(Bidirectional(LSTM(50),merge_mode='concat'))
        model.add(Dropout(0.5))
        model.add(Dense(self.labels_category))
        model.add(Activation('softmax'))#relu
        model.compile(loss='categorical_crossentropy',
                     optimizer='adam', 
                     metrics=['accuracy'])
        model.summary()
        return model
    
 
    def train(self,X_train, y_train,X_test, y_test,n_epoch=5 ):
        self.model.fit(X_train, y_train, batch_size=32, epochs=n_epoch,
                      validation_data=(X_test, y_test))
        self.model.save('sentiment.h5')   
        
 
    def predict(self,model_path,new_sen):
        model = self.model
        model.load_weights(model_path)
        new_sen_list = jieba.lcut(new_sen)
        sen2id =[ self.vocab.get(word,0) for word in new_sen_list]
        sen_input = keras.preprocessing.sequence.pad_sequences([sen2id], maxlen=self.maxlen)
        res = model.predict(sen_input)[0]
        return np.argmax(res)
 
 
senti = Sentiment(w2id,embedding_weights,100,200,2)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 100)          3735700   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100)               60400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total params: 3,796,302
Trainable params: 3,796,302
Non-trainable params: 0
_________________________________________________________________


In [18]:
senti.train(x_train,y_train, x_val ,y_val,5)

/Users/fgd/.conda/envs/keras-nlp/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 10640 samples, validate on 2660 samples
Epoch 1/5
10640/10640 [==============================] - 62s 6ms/step - loss: 0.3028 - accuracy: 0.8765 - val_loss: 0.2086 - val_accuracy: 0.9132
Epoch 2/5
10640/10640 [==============================] - 58s 5ms/step - loss: 0.1777 - accuracy: 0.9315 - val_loss: 0.1926 - val_accuracy: 0.9226
Epoch 3/5
10640/10640 [==============================] - 59s 6ms/step - loss: 0.1224 - accuracy: 0.9554 - val_loss: 0.1758 - val_accuracy: 0.9267
Epoch 4/5
10640/10640 [==============================] - 60s 6ms/step - loss: 0.0833 - accuracy: 0.9702 - val_loss: 0.1827 - val_accuracy: 0.9320
Epoch 5/5
10640/10640 [==============================] - 60s 6ms/step - loss: 0.0554 - accuracy: 0.9810 - val_loss: 0.2129 - val_accuracy: 0.9305


In [24]:
senti.predict("./sentiment.h5","不太行")

0